# Exercise 2

# Transform the recorded robot positions into a Self-Organizing Map (SOM)

--------------------------------------

--------------------------------------
### Initialization: Get data from solved Excercise 1
The goal of this exercise is to generate a Self-Organizing Map based on the recorded robot positions of Exercise 1.
If you didn't complete Exercise 1, clone and launch the experiment *"Exercise 2: Transform robot positions ... (SOM)"* of the [Neurorobotics platform](http://148.187.97.48/#/esv-private) to generate the file __*robot_positions.csv*__. Then follow the instructions below.

__Instructions:__
- Open __Exercise 1__ in your Collab's space.
- Move to the part __*C. "Record the robot positions"*__.
- Execute the commands of Section C using __*Exersice_2_i*__ from the list of NRP experiment when launching a simulation.


--------------------------------------------

--------------------------------------------

### Content:
- <font size="4">**[A. Installation of utils and download of storage files](#A)**</font></br>
>- <font size="3">[A.1. Install SOM utils](#A1)</font></br>
>- <font size="3">[A.2. Download csv input file from this Collab's storage](#A2)</font></br>
- <font size="4">**[B. Complete the SOM implementation](#B)**</font></br>
>- <font size="3">[B.1. SOM description](#B1)</font></br>
>- <font size="3">[B.2. SOM function (TO DO) ](#B2)</font></br>
>- <font size="3">[B.3. Upload SOM function into the storage](#B3)</font></br>
- <font size="4">**[C. SOM simulation](#C)**</font></br>
>- <font size="3">[C.1. Perform SOM training (editable) ](#C1)</font></br>
>- <font size="3">[C.2. Save SOM training result](#C2)</font></br>
- <font size="4">**[D. Result evaluation](#D)**</font></br>
- <font size="4">**[E. Submission and grading ](#E)**</font></br>

---------------------------------------------------

---------------------------------------------------
## A. Installation of SOM utils and download of storage files<a id='A'></a>

### A.1. Install SOM utils <a id='A1'></a>
Run the cell below to install SOM utils.

In [ ]:
# Install SOM utils
from IPython.display import clear_output
! pip uninstall epflx_robox_nrp_utils -y
! pip install --user --force-reinstall git+https://github.com/HBPNeurorobotics/EPFLx-RoboX-Neurorobotics-utils#egg=epflx_robox_nrp_utils
! pip install --upgrade "hbp-service-client==1.1.1"
clear_output()

### A.2. Download the csv input file from Collab storage <a id='A2'></a>
Run the cell below to download the input data.

In [ ]:
# Download the default robot_positions.csv file from this Collab's Storage
clients = get_hbp_service_client()
collab_path = get_collab_storage_path()
filename = raw_input("Enter the name of the data file you want to use for SOM simulation:")
if not filename: 
    filename = 'robot_positions.csv'
clear_output()
from os import path
clients.storage.download_file(path.join(collab_path, filename), './robot_positions.csv')

## Utility function to save files into Collab storage

In [ ]:
# Utility function used to save files into your Collab storage

"""
Save a file to the Collab storage

:param filepath: relative path to the file
:param filetype: type of the file, e.g., 'text/csv' or 'text/x-python'
:param remove: remove the file of the Jupyter notebook space if True 
"""
def save_to_collab_storage(filename, filetype, remove=False):
    from os import path
    clients = get_hbp_service_client()
    collab_path = get_collab_storage_path()
    filename = path.basename(filepath)
    # if you create new file
    if os.path.isfile(filename):
        # if old version of the file exists
        filepath = os.path.join(collab_name, filename)
        if clients.storage.exists(filepath): 
            clients.storage.delete(filepath)
        pydata = clients.storage.upload_file(filename, filepath, filetype)
        # remove 
        if remove:
            os.remove(filename)

---------------------------------------------------

---------------------------------------------------
## B. Complete the SOM implementation <a id='B'></a>

### B.1. SOM description <a id='B1'></a>
****

**Exercise goal:**<br>
Your task is to implement a Self-Organizing Map (SOM) that represents the robot environment. Mapping the environment to a 2D space makes visualization easy and hence helps you control the performance of the algorithm by immediate eye inspection.

**Input:** Coordinates of the robot positions *given by Exercise 1* (*'robot_positions.csv'*).<br>

**Output:** Self-Organizing Map (*'SOM_data_lattice.csv'*).<br>
****

#### The SOM algorithm
*<font color=green>**while**</font> (`N_trials` are not done):*<br>
> <font color=blue>*# TRIAL: `run_trial()`*</font><br>
> *mix your dataset: `datamix()` - an order*<br>
> *<font color=green>**while**</font> (not all points are processed):*<br>
>> *<font color=blue># EPISODE: `run_episode()`*</font><br>
>> *take next point according to an order*<br>
>> *calculate `distance()`*<br>
>> *define the `winner()`*<br>
>> *calculate `neighborhood_factor()`*<br>
>> *update SOM-lattice by `update_lattice()`*<br>

****

**Additional functions:**<br>
 - **`datamix()`:** generate new order of robot positions; <font color=green>**return**</font> *__array__ of new order*.<br>
 - **`distance()`:** the Euclidean distance between a data point and a neuron; <font color=green>**return**</font> *__array__ of distance to all neurons of SOM*.<br>
 - **`winner()`:** the winning neuron; <font color=green>**return**</font> *coordinates **[x,y]** of the winning neuron*.<br>
 - **`neighborhood_factor()`:** the Gaussian neighborhood function; <font color=green>**return**</font> *__matrix__ of neighborhood factors for each neuron to the current winning neuron*.<br>
 - **`update_lattice()`:** update the SOM-lattice according to SOM learning rule; <font color=green>**return**</font> *__nothing__*.<br>

**Variable parameters:**<br>
 - **`eta()`:** the leaning-rate parameter; <font color=green>**return**</font> *updated __eta__ value*.<br>
 - **`sigma()`:** the exponential decay parameter; <font color=green>**return**</font> *updated __sigma__ value*.<br>
****

**Global variables initialization (<font color=blue>default</font>, <font color=red>yours</font>):**<br>
 - <font color=blue>**self.video:**</font> the visualization trigger (False - no, True - yes).<br>
 - <font color=blue>**self.csv_file:**</font> the input file (*'NRP_data_robot_positions.csv'*).<br>
 - <font color=blue>**self.pos:**</font> the data from input file (*'NRP_data_robot_positions.csv'*).<br>
 - <font color=blue>**self.lattice:**</font> the initial and randomly generated SOM.<br>
 - <font color=blue>**self.trial:**</font> the index of the current trial (initial value is 0).<br>
<br>
 - <font color=red>**self.Nn:**</font> SOM size [_Nn_ x _Nn_].<br>
 - <font color=red>**self.eta_0:**</font> the initial value of the leaning-rate parameter. <br>
 - <font color=red>**self.sigma_0:**</font> the initial value of the exponential decay parameter<br>
 - <font color=red>**self.tau:**</font> the time constant of the SOM algorithm.<br>
 - <font color=red>**self.N_trials:**</font> the number of trials during learning.<br>
 - <font color=red>**self.lattice_limit:**</font> the range of initial values to initialize the SOM-lattice.<br>
****

<font color=red>**TO DO:**</font><br>
**1)** Initialize the <font color=red>constant parameters (yours)</font>.<br>
**2)** Implement the <font color=red>SOM training stages</font>: trial and episode.<br>
**3)** Implement the <font color=red>additional functions</font>.<br>
**4)** Implement the <font color=red>variable parameters</font>.<br>
****

<font color=red>**IMPORTANT:**</font><br>
**1)** The first line of the SOM implementation template: `%%writefile SOM_solution.py`.<br> Please, replace _'Surname'_ and _'Name'_ with your own Surname and Name. Don't uncomment this line until you are ready to submit.<br>
**2)** Please, pay attention to the fact that the grading of your implementation will be executed <font color=red>**only with SOM 12x12**</font>. Nevertheless, you can use other sizes of SOM to see how it works. For example, we will ask you to use a bigger lattice for Exercise 3 (reinforcement learning implementation).<br>
**3)** Please, keep in mind that the grading process has a time limit. <font color=red>**This time limit is 90 seconds**</font>.<br>
**4)** Once you are ready to submit, please, uncomment the first line of your script (`%%writefile SOM_solution.py`) and run the cell containing the script. This way your script will be saved into this Jupyter notebook space as a file with the given name. **Then, submit this file.**

****
****

### B.2. SOM function (TO DO) <a id='B2'></a>

In [ ]:
#%%writefile SOM_solution.py 

# class SOM (SOM_solution)
import numpy as np
import random
import time

from IPython import display


class SOM():

	#############################################################################################################
	#        Self-Organizing Map: map of the environment based on the positions visited by the robot        #
	#############################################################################################################
    
	# function to initialize class SOM
	def __init__(self, video=0, csv_file='robot_positions.csv', Nn=None):
		from epflx_robox_nrp_utils.SOM.SOM_additional import SOM_additional
		self.somad = SOM_additional()
		self.init_parameters()
		if(Nn!=None): self.Nn = Nn
		self.video = video
		self.csv_file = csv_file
		self.lattice = np.random.uniform(self.lattice_limit[0],self.lattice_limit[1],(self.Nn,self.Nn,2))
        
	# function to run SOM training
	def run_som(self):       
		T = time.time()
		self.pos = self.somad.load_data(self.csv_file)
		self.trial = 0
		while(self.trial < self.N_trials):
			self.run_trial()
			self.trial += 1
			self.somad.save_lattice(self.lattice,self.Nn)
			if(self.video): self.somad.visualization(self.lattice,self.Nn,self.eta(),self.sigma(),self.trial)
		display.clear_output(wait=True)
		print 'Done. Simulation time is ', time.time()-T, '(s).'
        
        
	"""======================================================================================================="""
	"""                                                TO DO                                                  """
	"""======================================================================================================="""
        
	##########################################
	"""         Constant parameters        """
	##########################################
        
	def init_parameters(self):
		self.Nn = 12
		self.eta_0 = 1.0
		self.sigma_0 = 1.0        
		self.tau_eta = 100.0
		self.tau_sigma = 100.0
		self.N_trials = 100.0
		self.lattice_limit = [0.0, 0.0]


	#########################################
	"""        SOM training stages        """
	#########################################

	def run_trial(self):
		mix = self.datamix();
		for i in range(self.pos.shape[0]):
			self.run_episode(mix, i)


	def run_episode(self, mix, i):
		pt = self.pos[mix[i]]
		d = self.distance(pt)
		[lx,ly] = self.winner(d)
		h = self.neighborhood_factor(lx[0],ly[0])
		self.update_lattice(h,pt)


	##########################################
	"""        Additional functions        """
	##########################################

	def datamix(self):
		mix = np.arange(self.pos.shape[0])
		np.random.shuffle(mix)
		return mix


	def distance(self, pt):
		return np.linalg.norm(self.lattice - pt,axis=2)**2


	def winner(self,d):
		return np.where(d == d.min())


	def neighborhood_factor(self,lx,ly):
		h = np.zeros((self.Nn,self.Nn))
		for i in range(self.Nn):
			for j in range(self.Nn):
				dist = (i-lx)**2 + (j-ly)**2
				h[i,j] = np.exp(-dist/(2*self.sigma()**2))
		return h


	def update_lattice(self,h,pt):
		for i in range(self.lattice.shape[2]):
			self.lattice[:,:,i] += self.eta()*h*((pt-self.lattice)[:,:,i])
    

	#########################################
	"""        Variable parameters        """
	#########################################

	def eta(self):
		return self.eta_0*(1-self.trial/self.tau_eta)


	def sigma(self):
		return self.sigma_0*(1-self.trial/self.tau_sigma)

****
### B.3. Upload the SOM function into your Collab storage<a id='B3'></a>
Once you are done and ready to submit your solution, please, follow next steps: </br>

- Move to [B.2. SOM function](#B2) and uncomment the first line  `%%writefile SOM_solution.py` ;
- Run the cell [B.2. SOM function](#B2). You should see  `Writing SOM_solution.py` just above; 
- Run the cell [B.3. Upload SOM function into the storage](#B3) to save this file in your Collab storage (check your Collab storage to see if the file is there).

Run the cell below to save your result into your Collab storage.

In [ ]:
save_to_collab_storage(filepath='SOM_solution.py', filetype='text/x-python', remove=True)

----------------------------------

----------------------------------
## C. SOM simulation <a id='C'></a>

### C.1. Perform SOM training (editable) <a id='C1'></a>
This cell runs the simulation of SOM training that uses [your script](#B2). You can visualize this process.
- *som = SOM(visualization)* # upload your SOM function [**visualization**: No - **0**, Yes - **1**];</br> 
- *som.run_som()* # run simulation.

In [ ]:
som = SOM(1)
som.run_som()

### C.2. Save SOM training result <a id='C2'></a>
The output of the SOM training is the SOM lattice. This lattice is represented with the red color on the visualization figure. 
The lattice data was saved into the file __*'SOM_data_lattice.csv'*__ which is now contained in this Jupyter notebook space. This commands upload the __*'SOM_data_lattice.csv'*__ into your Collab storage. 

Run the cell below and then check in your Collab storage if the __*'SOM_data_lattice.csv'*__ has been saved.

In [ ]:
save_to_collab_storage(filepath='SOM_data_lattice.csv', filetype='text/csv', remove=False)

---------------------------------------------------------------------

---------------------------------------------------------------------
## D. SOM evaluation <a id='D'></a>

Run the next cell to get a score for your implementation. This score is an average distance between data points (blue dots) and the centers of clusters (red dots) represented as a lattice. The goal of SOM training is to minimize this distance as much as possible.

In [ ]:
# Evaluation of the SOM training
from epflx_robox_nrp_utils.SOM.SOM_evaluation import SOM_evaluation
somev = SOM_evaluation()
var, N = somev.run_evaluation()
print "Your score is ", var, "with SOM size (", N, "x", N,")."

## E. Submission and grading
Run the next cell to submit your SOM implementation. Each submission is automatically tested and evaluated. Only the last submission will be taken into account for your grade.

In [ ]:
clients = get_hbp_service_client()
collab_path = get_collab_storage_path()
print("Retrieving your HBP OIDC token ...")
token = str(oauth.get_token())
print("token retrieved!")
submission_info = {
    'subheader': 'Exercise 2',
    'filepath': 'SOM_function.py',
    'token': token,
    'collab_path': collab_path,
    'clients_storage': clients.storage
}
from epflx_robox_nrp_utils.submission_manager.submission_widget import display_submission_widget
display_submission_widget(submission_info)